In [1]:
from cipher_operations import *
import pandas as pd
import numpy as np
import time

HE = init_scheme()

In [2]:
data = pd.read_csv('data.csv')
data

,Bankrupt,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


In [3]:
enc_data = encrypted_data(HE, data)
enc_data

{'116532cd6f61d0e2bfbaa7b445591516461e466b43ab6f36976714e58bfc0828': (<Pyfhel Ciphertext at 0x7f13cbac0040, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 'a865215a3bcec1654dec8cac5316f5f469e77c144e2455abe7db070e873115e6': (<Pyfhel Ciphertext at 0x7f13cbac04a0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 'ad981008f3abfaae3298f9436b451fc76f7b1dbfac0f10c8963eaff710eb64b2': (<Pyfhel Ciphertext at 0x7f13cbac0630, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 '77b63d7feb10a60863fbacace7117d1cae05b6559bcb925b9ebead0eca4b096b': (<Pyfhel Ciphertext at 0x7f13cbac07c0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 'd549c6993df52fad571c185381b3e1315f431641abeffc80386757e68cbc94ce': (<Pyfhel Ciphertext at 0x7f13cbac09a0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>,
  6819),
 '7b7b0e1b8ffe25afe2575f62927e8e4701dba46ee38d7572a9949f83d0bdb966': (<Pyfhel Ciphertext at 0x7f13cbac19e0, scheme=ckks, size=2/2, scale_bits=30, mod_level

In [6]:
operation_map = {
    'sum' : (np.sum, cipher_sum),
    'avg' : (np.mean, cipher_average),
    'ip': (np.dot, cipher_inner_product),
    'cov' : (np.cov, cipher_covariance),
}

In [8]:
for op_name, (np_op, cipher_op) in operation_map.items():
    print(f'{op_name} performance')
    for column_name in data.columns:
        raw_vector = data[column_name]
        enc_vector, size = enc_data[hash_column_name(column_name)]

        print(column_name)
        print('size:', size)


        start_time = time.perf_counter()
        if op_name in ('sum', 'avg'):
            raw_result = np_op(raw_vector)
        elif op_name == 'cov':
            raw_result = np_op(raw_vector, raw_vector)[0][0]
        else:
            raw_result = np_op(raw_vector, raw_vector)
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        print('unencrypted operation:', elapsed)

        start_time = time.perf_counter()
        if op_name in ('sum', 'avg'):
            cipher_result = cipher_op(enc_vector, size)
        else:
            cipher_result = cipher_op(enc_vector, enc_vector, size)
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        print('encrypted operation:', elapsed)

        off = raw_result - cipher_result.decrypt()[0]
        print('operation result difference:', off)

        print()

sum performance
Bankrupt
size: 6819
unencrypted operation: 0.0005548089975491166
encrypted operation: 0.4492223729903344
operation result difference: -0.0001280731727604234

 ROA(C) before interest and depreciation before interest
size: 6819
unencrypted operation: 0.00019510100537445396
encrypted operation: 0.461638449007296
operation result difference: 0.0005724633997488127

 ROA(A) before interest and % after tax
size: 6819
unencrypted operation: 0.00018506198830436915
encrypted operation: 0.403405836012098
operation result difference: 0.0009067529504136473

 ROA(B) before interest and depreciation after tax
size: 6819
unencrypted operation: 0.00019085299572907388
encrypted operation: 0.40835918400262017
operation result difference: -0.0016686280996509595

 Operating Gross Margin
size: 6819
unencrypted operation: 0.0001867239916464314
encrypted operation: 0.4230991199874552
operation result difference: 0.0017857293059933

 Realized Sales Gross Margin
size: 6819
unencrypted operation: